In [1]:
import tweepy
import json
from pymongo import MongoClient
from collections import Counter
import logging
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import community
import seaborn as sns
from fastprogress import master_bar, progress_bar

#Logger
logging.basicConfig(filename='Methodology.log', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logging.getLogger().addHandler(ch)

In [ ]:
################
# RESTART DATA #
################

H = nx.read_gml("Cat-CI-Complete.gml")

print("Nombre de nodes: {}".format(H.number_of_nodes()))
print("Nombre d'arestes: {}".format(H.number_of_edges()))

with open('Cat-community_dict.json', 'r') as fp:
    community_dict = json.load(fp)
    
len(community_dict)

In [ ]:
H.remove_edges_from(nx.selfloop_edges(H))
print("Nombre de nodes: {}".format(H.number_of_nodes()))
print("Nombre d'arestes: {}".format(H.number_of_edges()))

H = nx.k_core(H, k=30)

print("Nombre de nodes: {}".format(H.number_of_nodes()))
print("Nombre d'arestes: {}".format(H.number_of_edges()))

In [ ]:
############################
# FORCE ATLAS 2            #
############################

from fa2 import ForceAtlas2

forceatlas2 = ForceAtlas2(
                        # Behavior alternatives
                        outboundAttractionDistribution=True,  # Dissuade hubs
                        linLogMode=False,  # NOT IMPLEMENTED
                        adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
                        edgeWeightInfluence=1.0,

                        # Performance
                        jitterTolerance=1.0,  # Tolerance
                        barnesHutOptimize=True,
                        barnesHutTheta=1.2,
                        multiThreaded=False,  # NOT IMPLEMENTED

                        # Tuning
                        scalingRatio=2.0,
                        strongGravityMode=False,
                        gravity=1.0,

                        # Log
                        verbose=True)

positions = forceatlas2.forceatlas2_networkx_layout(H, pos=None, iterations=500)

In [ ]:
for i in range (20):
    print("STEP: " + str(i))
    positions = forceatlas2.forceatlas2_networkx_layout(H, pos=None, iterations=500)
    print(positions["vox_es"])
    if positions["vox_es"][0] > 3500:
        break

In [ ]:
########################
# SAVE FA2 POSITIONS   #
########################

print(positions)

with open('Cat-fa2_positions_dict.json', 'w') as fp:
    json.dump(positions, fp)

In [ ]:
########################
# READ FA2 POSITIONS   #
########################

with open('Cat-fa2_positions_dict.json', 'r') as fp:
    positions = json.load(fp)       

print(len(positions))
print(positions)

In [ ]:
#################################
# VISUALIZE (WIH PARTY NAMES)   #
#################################

COMMUNITIES_LIST = ["VOX", "JXCAT", "CUP", "COMUNS", "PSC", "ERC", "PP", "CS"]
#COMMUNITIES_LIST = ["VOX", "POR_AND", "PSOE", "PP", "ADELANTE_AND", "CS"]
#COMMUNITIES_LIST = ["VOX", "PODEMOS", "PSOE", "PP", "MAS_MAD", "CS"]

COLORS_LIST = ["darkgreen", "lightgreen", "yellow", "purple", "red", "orange", "blue", "darkorange"]

def community_color(node_name):
    community = community_dict[node_name]
    index = COMMUNITIES_LIST.index(community)
    color = COLORS_LIST[index]
    return color

def scale_position(position):
    scaled_position = position/1000 + 5
    return scaled_position

# Visualize embeddings in 2D space with node labels
plt.figure(figsize=(12, 10))
#plt.scatter(embedding[:, 0], embedding[:, 1], s=1)
#plt.scatter(positions.values(), c='blue', alpha=0.7)


'''
#Only label top10 in-degree nodes
indeg = G.in_degree(weight='weight')
sorted_indeg = sorted(indeg, key=lambda i: i[1], reverse=True)[:10]
print("Usuaris amb més grau d'ENTRADA:")
print(sorted_indeg)
sorted_indeg_names = [i[0] for i in sorted_indeg]
'''

ideological_position_dict ={}

# Add node labels
#party_names = ['vox_es', 'PSOE', 'populares', 'PODEMOS', 'EnComu_Podem', 'CiudadanosCs', 
#               'socialistes_cat', 'Esquerra_ERC', 'JuntsXCat', 'cupnacional']
party_names = ['vox_es', 'populares', 'EnComu_Podem', 'CiudadanosCs', 'socialistes_cat', 
               'Esquerra_ERC', 'JuntsXCat', 'cupnacional']
for node_name, position in positions.items():
    if position[0]<5000 and position [0]>-5000:
        if position[1]<5000 and position [1]>-5000:
            #plt.scatter(position[0], position[1], c='blue', s=1)
            plt.scatter(scale_position(position[0]), scale_position(position[1]), c=community_color(node_name), s=1)
            if node_name in party_names:
                print(node_name)
                print(position)
                ideological_position_dict [node_name] = [scale_position(position[0]), scale_position(position[1])]
                #plt.plot(position[0], position[1], c='green', marker="X")
                #plt.text(position[0], position[1], node_name, fontsize=8, c='red')
                if node_name == 'socialistes_cat':
                    plt.text(scale_position(position[0]), scale_position(position[1]), node_name, fontsize=12, c='blue')
                    plt.plot(scale_position(position[0]), scale_position(position[1]), c='blue', marker="X")
                else:
                    plt.text(scale_position(position[0]), scale_position(position[1]), node_name, fontsize=12, c='red')
                    plt.plot(scale_position(position[0]), scale_position(position[1]), c='red', marker="X")
        

#plt.xlabel('X Dimension')
#plt.xlabel('Y Dimension')
plt.title('Catalonia - Visualization of RT Network')
#plt.savefig("CAT-PARTIES.png")
plt.show()

In [ ]:
print(ideological_position_dict)

In [ ]:
#################################
# VISUALIZE (WIH CANDIDATES NAMES)   #
#################################

COMMUNITIES_LIST = ["VOX", "JXCAT", "CUP", "COMUNS", "PSC", "ERC", "PP", "CS"]
COLORS_LIST = ["darkgreen", "lightgreen", "yellow", "purple", "red", "orange", "blue", "darkorange"]

def community_color(node_name):
    community = community_dict[node_name]
    index = COMMUNITIES_LIST.index(community)
    color = COLORS_LIST[index]
    return color

# Visualize embeddings in 2D space with node labels
plt.figure(figsize=(12, 10))
#plt.scatter(embedding[:, 0], embedding[:, 1], s=1)
#plt.scatter(positions.values(), c='blue', alpha=0.7)


'''
#Only label top10 in-degree nodes
indeg = G.in_degree(weight='weight')
sorted_indeg = sorted(indeg, key=lambda i: i[1], reverse=True)[:10]
print("Usuaris amb més grau d'ENTRADA:")
print(sorted_indeg)
sorted_indeg_names = [i[0] for i in sorted_indeg]
'''

# Add node labels
candidate_names = ['Igarrigavaz', 'LauraBorras', 'mariadolorsa', 'jessicaalbiach', 
                   'salvadorilla', 'perearagones', 'alejandroTGN', 'carrizosacarlos']
for node_name, position in positions.items():
    if position[0]<6000 and position [0]>-6000:
        if position[1]<6000 and position [1]>-6000:
            #plt.scatter(position[0], position[1], c='blue', s=1)
            plt.scatter(position[0], position[1], c=community_color(node_name), s=1)
            if node_name in candidate_names:
                print(node_name)
                print(position)
                #plt.plot(position[0], position[1], c='green', marker="X")
                #plt.text(position[0], position[1], node_name, fontsize=8, c='red')
                if node_name == 'salvadorilla':
                    plt.text(position[0], position[1], node_name, fontsize=8, c='blue')
                else:
                    plt.text(position[0], position[1], node_name, fontsize=8, c='red')
                

plt.xlabel('X Dimension')
plt.xlabel('Y Dimension')
plt.title('Visualization of RT Network')
#plt.savefig("CAT-CANDIDATES.png")
plt.show()

In [ ]:
###############################
# SAVE IDEOLOGICAL POSITION   #
###############################

print(ideological_position_dict)

with open('Cat-ideological_position_dict.json', 'w') as fp:
    json.dump(ideological_position_dict, fp)